In [1]:
!pip install -q transformers
!pip install sentencepiece
!pip install -q prenlp


     |████████████████████████████████| 2.3MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 48.2MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 
     |████████████████████████████████| 1.2MB 8.4MB/s 
     |████████████████████████████████| 133kB 13.8MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 19.4MB 8.5MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 450kB 44.9MB/s 


In [2]:
import os
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from tqdm.notebook import tqdm
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from prenlp.data import Normalizer


#Load dataset

In [7]:
df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/offLabel.xlsx')
df_test.head()

,tweets
0,@USER اما انت تقعد طول عمرك لا مبدا ولا راي ث...
1,@USER @USER بتخاف نسوانك يزعلوا ولا ايه 😂 اه ...
2,RT @USER: يا عـسانـى نـبـقى يا عـمري حـبايـب ...
3,RT @USER: باقي البيان وينو ما شفنه يا برهان <...
4,@USER @USER اللهم انت الشافي المعافي اشفيه وج...


#Preprocessing

In [ ]:
normalizer = Normalizer(url_repl='[URL]', tag_repl='[TAG]', emoji_repl='[EMOJI]', email_repl='[EMAIL]', tel_repl='[TEL]', image_repl='[IMG]')

In [ ]:
text_cleaning_re = "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]|[.#،<>:@,\\-_”“٪ًَ]"
def preprocess(text):
  #remove tashkeel
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  text = re.sub(p_tashkeel,"", str(text))
  # text = text.replace('وو', 'و')
  # text = text.replace('يي', 'ي')
  # text = text.replace('اا', 'ا')
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  #remove longation
  text = re.sub(r'(.)\1+', r'\1\1', str(text)) 
  text = re.sub("[إأآا]", "ا", str(text))
  text = re.sub("ى", "ي", str(text))
  # text = re.sub("ؤ", "ء", text)
  # text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", str(text))
  text =normalizer.normalize(text)

  tokens = []
  for token in text.split():
    # if token not in stop_words:
      tokens.append(token)
  return " ".join(tokens)

In [ ]:
x_test = df_test.tweets
x_test = x_test.apply(lambda x: preprocess(x))
x_test[:10]

0    اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا ...
1    بتخاف نسوانك يزعلوا ولا ايه [EMOJI] اه يا هلفو...
2    يا عـسانـي نـبـقي يا عـمري حـبايـب وحـبنـا يكب...
3    باقي البيان وينو ما شفنه يا برهان ورينا يا بره...
4    اللهم انت الشافي المعافي اشفيه وجميع مرضي المس...
5    ااه يا غيث يا حبيبنا لو تعرف حجم الفراغ الي تر...
6    يا حب يا دنيا جديده يا احلي يا احلي ابيات القص...
7    صباح الاربعاء هذي السنين ستنقضي وستفرحين وستضح...
8    عاوز اوجه رساله ل الدكاتره و اني بعرف امارس ال...
9               حبيبي يا هندسه وانت طيب يا ابا [EMOJI]
Name: tweets, dtype: object

In [4]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [5]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [8]:
classes = ["Hate","NOT_Hate"]
pred = []
for tweet in df_test.tweets:
  encoded_input = tokenizer(tweet, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  if((scores[0] > scores[1]) & (scores[0] >scores[2])  ):
    pred.append(classes[0])
  else:
    pred.append(classes[1])
  # y_pred.append(int(scores[0] > scores[1] && scores[0] >scores[2]  ))
  # pred1.append(int(paraphrase_results[0] > 0))
  # pred.append(scores)
  # y_pred = np.argmax(pred, axis=1)
  # pred.append(int(round(paraphrase_results[0] * 100)))
pred[:10]

['Hate',
 'Hate',
 'NOT_Hate',
 'Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'NOT_Hate',
 'Hate',
 'NOT_Hate']

# Confusion matrix

In [9]:
#Label Encoding to convert labels to integer values # Actual label

y_test=df_test_labels.labels
LE = LabelEncoder()
# Actual label
y_test=LE.fit_transform(y_test)
print(y_test)
# Predicted label
y_pred=1-LE.fit_transform(pred)
print(y_pred)

[1 1 0 ... 0 1 1]
[1 1 0 ... 0 0 1]


In [10]:
print("Accuracy", accuracy_score(y_test, y_pred))

print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

Accuracy 0.814

Confusion matrix
 [[1316  282]
 [  90  312]]

Classification report
               precision    recall  f1-score   support

           0       0.94      0.82      0.88      1598
           1       0.53      0.78      0.63       402

    accuracy                           0.81      2000
   macro avg       0.73      0.80      0.75      2000
weighted avg       0.85      0.81      0.83      2000

